In [1]:
from src.dlq_risk_calculation import LoanDlqRisks
import pandas as pd
import numpy as np
import pickle
import altair as alt

In [13]:
df = pd.read_csv('dashcomponents/loan_data.csv')
with open('dashcomponents/weights.pkl', 'rb') as file:
    weights = pickle.load(file)

In [14]:
df['Zip Code Short'] = df['Zip Code Short'].fillna(0).astype(int).astype(str)

In [15]:
interest_rate = 2
total_loan_amount=10000
loan_to_value=25
fico_score=780
dti_ratio=55
zipcode=0

In [16]:
dlqr = LoanDlqRisks(
            interest_rate,
            total_loan_amount,
            loan_to_value,
            fico_score,
            dti_ratio,
            zipcode
        ).get_relative_stats(ref_table=df).generate_risk_summary(weight_params=weights)

Too few samples for zipcode starting 0, using all data instead


/Users/xiaoma/Desktop/gitrepo/ucbmids_w209_summer2023_fanniemae/fannie-mae-viewer/src/dlq_risk_calculation.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_plottb['value_direc'] = temp_plottb['values'].apply(lambda x: 'high' if x>0 else 'low')
/Users/xiaoma/Desktop/gitrepo/ucbmids_w209_summer2023_fanniemae/fannie-mae-viewer/src/dlq_risk_calculation.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_plottb['des'] = temp_plottb['value_direc'] + ' ' + temp_plottb['labels']


In [17]:
dlqr.combined

alt.VConcatChart(...)

In [ ]:
if dlqr.risk_score<0:
    temp_plottb = dlqr.plot_tb[dlqr.plot_tb.directions==True]
    temp_plottb['value_direc'] = temp_plottb['values'].apply(lambda x: 'high' if x>0 else 'low') 
    temp_plottb['des'] = temp_plottb['value_direc'] + ' ' + temp_plottb['labels']
    description_str = ', '.join(temp_plottb['des'].values)
    subtitle = f"""
    This loan is rated as {dlqr.risk_cat} because it has {description_str} comparing to loans from the same area
    """
else:
    temp_plottb = dlqr.plot_tb[dlqr.plot_tb.directions==False]
    temp_plottb['value_direc'] = temp_plottb['values'].apply(lambda x: 'high' if x>0 else 'low') 
    temp_plottb['des'] = temp_plottb['value_direc'] + ' ' + temp_plottb['labels']
    description_str = ', '.join(temp_plottb['des'].values)
    subtitle = f"""
    This loan is rated as {dlqr.risk_cat} because it has {description_str} comparing to loans from the same area
    """

In [ ]:
chart_title = alt.TitleParams(
    f"Why is this loan catagorized as {dlqr.risk_cat}?",
    subtitle=[subtitle],
)

diag = alt.Chart(
    dlqr.plot_tb[dlqr.plot_tb.directions==(dlqr.risk_score<0)], title=chart_title
).mark_bar(
).encode(
    x=alt.X('labels', 
            title='Loan Attributes',
            axis=alt.Axis(labelAngle=0)
    ),
    y=alt.Y(
        'values:Q', 
        title='Percentage Delta Vs Area Average',
        axis=alt.Axis(format='%'),
        scale=alt.Scale(domain=[-1,1])
    ),
    color=alt.condition(
        alt.datum.directions,
        alt.value("green"),  # The positive color
        alt.value("red")  # The negative color
    )
).properties(
    width=800,
    height=300
)

In [ ]:
dlqr.risk_score_scale_chart

In [ ]:
alt.vconcat(dlqr.risk_score_scale_chart, diag)